<h1><b>Ο αλγόριθμος Logistic Regression</b></h1>
<p align="justify">Στην άσκηση αυτή θα μελετήσετε τον αλγόριθμο <i>logistic regression</i>, κατασκευάζοντας ένα σύντομο πρόγραμμα. Στην άσκηση αυτή θα χρησιμοποιήσετε τη βιβλιοθήκη της <i>Python Scikit-Learn</i>. Για τη διευκόλυνσή σας παρέχονται οι δηλώσεις των βιβλιοθηκών που θα χρησιμοποιήσετε καθώς και εντολές με κενά. Συγκεκριμένα, θα κατασκευάσετε έναν <i>ταξινομητή Spam μηνυμάτων SMS</i>. Τα δεδομένα που θα χρησιμοποιήσετε για την εκπαίδευση και την επικύρωση του μοντέλου, καθώς και πληροφορίες για αυτά μπορούν να βρεθούν <a href="https://archive.ics.uci.edu/ml/datasets/sms+spam+collection"><i>εδώ</i></a>.</p>
<p align="justify">Καλείστε να αναπτύξετε ένα πρόγραμμα, το οποίο:</p>
<ul>
<li>Θα φορτώνει τα δεδομένα από το αρχείο <i>.csv</i>.</li>
<li>Θα διαχωρίζει με τυχαίο τρόπο τα δεδομένα που παρέχονται σε δεδομένα εκπαίδευσης <i>(training set)</i> και δεδομένα για τον υπολογισμό της ακρίβειας του μοντέλου <i>(test set)</i>.</li>
<li>Θα πραγματοποιεί προεπεξεργασία στα δεδομένα χρησιμοποιώντας τη μέθοδο <i>TfidVectorizer</i> της βιβλιοθήκης <i>Scikit-Learn</i>. Περισσότερες πληροφορίες για τη μέθοδο <i>TfidVectorizer</i>, που περιλαμβάνεται στις δηλώσεις του προγράμματος, μπορούν να βρεθούν <a href="https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.TfidfVectorizer.html"><i>εδώ</i></a>. Να σημειωθεί πως η μέθοδος <i>TfidVectorizer</i> της <i>Scikit-Learn</i> αφαιρεί και τα σημεία στίξης, καθώς επεξεργάζεται το dataset.</li>
<li>Θα εκπαιδεύει το μοντέλο <i>logistic regression</i>.
<li>Θα υπολογίζει την ακρίβειά του πάνω στο <i>test set</i>.</li>
</ul>
<p align="justify">Για ποιους λόγους πιστεύετε ότι δεν είναι κατάλληλη η εφαρμογή της μεθόδου <i>linear regression</i> στο συγκεκριμένο πρόβλημα;</p>

<font color='#486393'>Τα σχόλια στα βήματα της παρούσας αναφοράς γράφονται σε αυτό το χρώμα, προκειμένου να διαφοροποιούνται από την εκφώνηση ή τις οδηγίες.</font>

<font color='#486393'>Όπως υποδεικνύει και το όνομά της, η γραμμική *παλινδρόμηση* είναι ένα μοντέλο παλινδρόμησης το οποίο στοχεύει στη βέλτιστη προσαρμογή μιας ευθείας (στην απλή περίπτωση) σε ένα σύνολο δεδομένων εκπαίδευσης με σκοπό την πρόβλεψη συνεχών τιμών. Για το λόγο αυτό, χρησιμοποιείται συχνά σε προβλήματα πρόβλεψης κόστους, θερμοκρασίας, κτλ. Στην προκείμενη περίπτωση διαθέτουμε ένα πρόβλημα ταξινόμησης, όπου στόχος είναι η εκπαίδευση ενός δυαδικού ταξινομητή, δηλαδή ενός μοντέλου που να προβλέπει 1 ή 0 (spam ή όχι spam). Έτσι, η γραμμική παλινδρόμηση δεν αποτελεί ένα κατάλληλο μοντέλο, σε αντίθεση με τη λογιστική παλινδρόμηση η οποία, παρά το παραπλανητικό της όνομα, είναι ένας ταξινομητής.</font>

In [1]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score

<font color='#486393'>Το πρώτο βήμα είναι η λήψη του αρχείου των δεδομένων και η μεταφόρτωσή του σε ένα pandas dataframe. Τα αρχικά labels των εγγραφών είναι ham για τα SMS που δεν αποτελούν spam και spam για τα υπόλοιπα, επομένως μετατρέπονται σε 0 και 1, αντίστοιχα.</font>

In [2]:
# load csv file
df = pd.read_csv("SMSSpamCollection", sep = '\t', header = None, names = ['label', 'SMS'])
df = df.replace(['ham', 'spam'], [0, 1])
df.head()

,label,SMS
0,0,"Go until jurong point, crazy.. Available only ..."
1,0,Ok lar... Joking wif u oni...
2,1,Free entry in 2 a wkly comp to win FA Cup fina...
3,0,U dun say so early hor... U c already then say...
4,0,"Nah I don't think he goes to usf, he lives aro..."


<font color='#486393'>Κατόπιν, πραγματοποιείται ο διαχωρισμός των δεδομένων σε δύο υποσύνολα: ένα υποσύνολο για την εκπαίδευση του ταξινομητή (train), καθώς και ένα για την αξιολόγησή του (test). Μιας και δεν υπάρχει ρητή οδηγία σε ό,τι αφορά την αναλογία του διαχωρισμού αυτού, επιλέγεται στο υποσύνολο αξιολόγησης να χρησιμοποιηθεί το 20% του συνόλου, ενώ το υπόλοιπο 80% αφιερώνεται στο μέρος της εκπαίδευσης. Στα παρακάτω, η επιλογή </font>`stratify` <font color='#486393'>γίνεται προκειμένου η αναλογία των δύο labels να είναι κοινή στα δεδομένα εκπαίδευσης και αξιολόγησης.</font>

In [3]:
# split dataset
X_train_raw, X_test_raw, y_train, y_test = train_test_split(df['SMS'], df['label'], stratify=df['label'], test_size=0.2, random_state=42)

<font color='#486393'>Το επόμενο βήμα είναι η προεπεξεργασία των δεδομένων μέσω του </font>`TfidfVectorizer` <font color='#486393'>της sklearn. Ο όρος **TF-IDF** σημαίνει **Term Frequency - Inverse Document Frequency** και χρησιμοποιείται στο πεδίο της ανάκτησης πληροφορίας ως μέτρο της σημασίας μιας λέξης σε ένα κείμενο. Συγκεκριμένα, η τιμή TF-IDF αυξάνεται κατ' αναλογία με τη συχνότητα εμφάνισής της σε ένα κείμενο και σταθμίζεται βάσει του πλήθους κειμένων στα οποία εμφανίζεται. Η στάθμιση αυτή πραγματοποιείται διότι υπάρχουν λέξεις με υψηλότερη a-priori πιθανότητα εμφάνισης, όπως για παράδειγμα τα άρθρα.</font>

<font color='#486393'>Όπως αναλύθηκε και στην εργαστηριακή αναφορά της Γραμμικής Παλινδρόμησης, είναι πολύ σημαντικό το μοντέλο προεπεξεργασίας να εκπαιδεύεται αποκλειστικά στα train δεδομένα και να εφαρμόζεται αυτούσιο στα test δεδομένα για να αποφευχθεί η μεταφορά πληροφορίας του θεωρητικά αγνώστου συνόλου δεδομένων στο τελικό μοντέλο.</font>

In [4]:
# vectorize data
vectorizer = TfidfVectorizer()
X_train = vectorizer.fit_transform(X_train_raw)
X_test = vectorizer.transform(X_test_raw)

<font color='#486393'>Στη συνέχεια, ορίζεται το μοντέλο λογιστικής παλινδρόμησης (υλοποίηση της sklearn) και εκπαιδεύεται στα χαρακτηριστικά που έχουν προκύψει από την προεπεξεργασία.</font>

In [5]:
# fit Logistic Regression model
model = LogisticRegression()
model.fit(X_train,y_train)

LogisticRegression()

<font color='#486393'>Το τελευταίο βήμα αποτελεί η αξιολόγηση του μοντέλου στα δεδομένα αξιολόγησης.</font>

In [6]:
# calculate accurary based on test set
acc = model.score(X_test,y_test)
print(f"Η ακρίβεια του ταξινομητή στα δεδομένα αξιολόγησης προκύπτει ίση με {acc*100:.2f}%.")

Η ακρίβεια του ταξινομητή στα δεδομένα αξιολόγησης προκύπτει ίση με 97.31%.


<font color='#486393'>Αξίζει να σημειωθεί στο σημείο αυτό πως η ακρίβεια αυτή καθ' αυτή επιλέγεται ως μέτρο αξιολόγησης επειδή αποτελεί ζητούμενο της άσκησης. Θεωρητικά, μια καταλληλότερη μετρική για την προκείμενη περίπτωση είναι το F1-Score, το οποίο αποτελεί τον αρμονικό μέσο των Precision και Recall, διότι το σύνολο των δεδομένων δεν είναι ισορροπημένο.</font>

In [7]:
SPAM = df[df['label']==1].count()[0]
print(f"Από το σύνολο των δεδομένων, το {100*SPAM/df.shape[0]:.2f}% αντιστοιχεί σε spam.")

Από το σύνολο των δεδομένων, το 13.41% αντιστοιχεί σε spam.


<font color='#486393'>Συγκεκριμένα, η αναλογία spam/not spam είναι μεγαλύτερη από 1:6. Ως προς αυτό το μέτρο, η απόδοση του ταξινομητή υπολογίζεται ως:</font>

In [8]:
y_pred = model.predict(X_test)
f1 = f1_score(y_test,y_pred)
print(f"Το F1-Score του ταξινομητή στα δεδομένα αξιολόγησης προκύπτει ίσο με {f1*100:.2f}%.")

Το F1-Score του ταξινομητή στα δεδομένα αξιολόγησης προκύπτει ίσο με 88.81%.
